In [ ]:
# Those packages are required to run the following code

!pip install langchain==0.1.0
!pip install langchain-core==0.1.10
!pip install langchain_openai==0.0.2
!pip install langchain-community==0.0.12
!pip install tiktoken==0.5.2
!pip install wikipedia==1.4.0
!pip install faiss-cpu==1.7.4

In [ ]:
import os
import langchain
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.vectorstores.utils import DistanceStrategy

# Fill in those variables before running the notebook

In [ ]:
endpoint_name = "<ENDPOINT_NAME>"
key = "<YOUR_KEY>" 
embeddings_deployment_name = "<YOUR_EMBEDDINGS_DEPLOYMENT_NAME>"

api_version="2023-07-01-preview"

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = key
os.environ["AZURE_OPENAI_ENDPOINT"] = f"https://{endpoint_name}.openai.azure.com/"

# Run RAG on Wikipedia

In [ ]:
llm = AzureChatOpenAI(
    api_version = "2023-07-01-preview", 
    azure_endpoint=f"https://{endpoint_name}.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version={api_version}")

In [ ]:
# check that the model is well-defined
llm.invoke('how are you?')

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embeddings_deployment_name,
    openai_api_version=api_version,
)

In [ ]:
wikipedia_docs = WikipediaLoader(query= 'Eurovision Song Contest 2022' , load_max_docs=1).load() # add your own Wikiepdia value
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=20, length_function=len)
splited_docs = text_splitter.split_documents(wikipedia_docs)

In [ ]:
db = FAISS.from_documents(documents=splited_docs, embedding=embeddings, distance_strategy=DistanceStrategy.COSINE)

In [ ]:
query = 'Who won?'
query_embedding = embeddings.embed_query(query)
print(query_embedding[:10])

In [ ]:
k=3
context = "\n\n".join([db.similarity_search(query=query, k=k, distance_strategy=DistanceStrategy.COSINE)[i].page_content for i in range(k)])
print(context)

In [ ]:
RAG_prompt_template = ("Answer the questino below based on the provided context:\n\n" 
                       "Question:\n{question}\n\n"
                       "Context:\n{context}\n\n")

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(RAG_prompt_template),
    ]
)

# Those lines are meant to print the final prompt string
messages = chat_template.format_messages(question=query,
                                         context=context)
print(messages[0].content)

In [ ]:
chain = chat_template | llm 
chain.invoke({"question": query, "context": context})